To dos:
- create a dataset with age and lon, lat and download it - CHECK
- specify a new location to save images on the server -  CHECK
- loop over the dataset - CHECK
 - plug in lat and lon into the api code below - CHECK
 - extract all images for that location - CHECK
 - name image according to age and its unique code after the last - CHECK

CNN
- rewatch simple CNN video how to extract age from name file - CHECK
- run a simple CNN and compare results - CHECK
- how to boost perfromance - HALF-CHECK - wait for reply from Nikola and start reading papers to do it yourself.
- How to increase image quality and coverage? - HALF-CHECK - wait for Nikolas reply on a code basis, read other papers on how they implement it.

Extent dataset:
- access Mapillary data either through API or the download you did
- repeat above excercise
- compare R2
- compare coverage of images between the two - are the duplicates?

Integration
- add a building ID to the kartaview key 
- Download images for buildings that do not have transaction data and train the algorithm on it
- use the images that have transaction data as test set - CAREFUL THO DUE TO SELECTION ISSUES.
- save the age predictions from this first step and add them as prediction variable into the matched GIS-RCA dataset as a variable using the building ID as key
- rerun the xgboost model on it and see how it performs
- in the best scenario, you can develop a multi-modal model that optimises the relational and image data together in one deep learning model, but this is REALLY OPTIONAL
- OPTIONAL: Does it make sense to run a computer vision excercise to predict architectural style and feed that into the relational data?

## Imports

In [23]:
import requests
import re
import json
import os
import pandas as pd
import sys
import numpy as np

# project lib
PROJECT_SRC_PATH = os.path.join( '/workspace/workspace/ufo-prediction', 'src-RCA-UFO')
sys.path.append(PROJECT_SRC_PATH)
import dataset

## Data

In [33]:
# Set a seed for reproducibility
np.random.seed(42)  # You can choose any number as your seed

# Define paths for data
path_data_NLD = os.path.join('/workspace/workspace/ufo-prediction', 'demo', 'df-NLD.pkl')
path_data_FRA = os.path.join('/workspace/workspace/ufo-prediction', 'demo', 'df-FRA.pkl')
path_data_ESP = os.path.join('/workspace/workspace/ufo-prediction', 'demo', 'df-ESP.pkl')
processed_df_NLD = "/workspace/workspace/ufo-prediction/demo/kartaview_key_NLD.csv"
processed_df_ESP = "/workspace/workspace/ufo-prediction/demo/kartaview_key_ESP.csv"
processed_df_FRA = "/workspace/workspace/ufo-prediction/demo/kartaview_key_FRA.csv"
processed_df_ALL = "/workspace/workspace/ufo-prediction/demo/kartaview_key.csv"

# Image directory paths
image_dir_map = {
    'NLD': '/workspace/workspace/ufo-prediction/image_data_NLD',
    'FRA': '/workspace/workspace/ufo-prediction/image_data_FRA',
    'ESP': '/workspace/workspace/ufo-prediction/image_data_ESP',
    'ALL': '/workspace/workspace/ufo-prediction/image_data'
}

# Ask for user input
country_code = input("Enter country code (FRA, NLD, ESP) or ALL: ").upper()

# Map user input to the correct path
path_data_map = {
    'NLD': processed_df_NLD,
    'FRA': processed_df_FRA,
    'ESP': processed_df_ESP,
    'ALL': processed_df_ALL
}

raw_data_path_map = {
'NLD': path_data_NLD,
'FRA': path_data_FRA,
'ESP': path_data_ESP
}
# Check if the input is valid
if country_code not in path_data_map:
    print("Invalid country code or specification. Please enter FRA, NLD, ESP, or ALL.")
else:
    processed_df_path = path_data_map[country_code]
    # Set directory based on country code
    current_directory = image_dir_map[country_code]

    # Check if the new directory exists, if not, create it
    if not os.path.exists(current_directory):
        os.makedirs(current_directory)
        print(f"Directory {current_directory} created.")
    else:
        print(f"Directory {current_directory} already exists.")

    # Process for ALL
    if country_code == 'ALL':
        if os.path.exists(processed_df_path):
            kartaview_keys = pd.read_csv(processed_df_path)
            print("Loaded processed DataFrame from", processed_df_path)
        else:
            print("Creating a new processed DataFrame for ALL")
            path_data_RCA = os.path.join(dataset.DATA_DIR, 'rca-ufo-merge_ALL.csv')
            df = pd.read_csv(path_data_RCA, encoding='latin1')
            kartaview_keys = df[['lon', 'lat','age_right', 'id', 'PropertyKey_ID']]
            kartaview_keys.to_csv(processed_df_path, index=False)
            

    # Process for FRA, NLD, ESP
    else:
        if os.path.exists(processed_df_path):
            kartaview_keys = pd.read_csv(processed_df_path)
            print("Loaded processed DataFrame from", processed_df_path)
        else:
            print(f"Creating a new processed DataFrame for {country_code}")
            df_path = raw_data_path_map[country_code]
            df = pd.read_pickle(df_path)
            print("Loaded DataFrame from", df_path)
            sampled_df = df.sample(n=100000, random_state=42)
            kartaview_keys = sampled_df[['lon', 'lat', 'age', 'id']].rename(columns={'age': 'age_right'})
            kartaview_keys.to_csv(processed_df_path, index=False)

print(kartaview_keys) 

Directory /workspace/workspace/ufo-prediction/image_data_NLD already exists.
Creating a new processed DataFrame for NLD
Loaded DataFrame from /workspace/workspace/ufo-prediction/demo/df-NLD.pkl
                   lon        lat  age_right                     id
97  4583413   4.738666  52.947502     1953.0  v0.1-NLD.9.15_1-24152
315 19789747  5.570947  51.770518     1974.0   v0.1-NLD.8.47_1-6310
455 3874217   5.117115  52.237031     1972.0   v0.1-NLD.9.59_1-6586
411 22572960  5.554535  52.013209     1950.0  v0.1-NLD.11.28_1-9596
15  32521724  5.206793  52.362430     1982.0   v0.1-NLD.2.1_1-85419
...                ...        ...        ...                    ...
269 18053623  5.787166  51.682889     1935.0    v0.1-NLD.8.41_1-176
123 42905501  6.715116  52.105296     1980.0   v0.1-NLD.4.19_1-4362
127 25117464  6.904889  52.745651     1984.0   v0.1-NLD.1.6_1-23122
373 26904799  5.630706  53.043482     1989.0   v0.1-NLD.3.25_1-5671
262 37334101  5.668091  53.212675     1974.0   v0.1-NLD.3.

In [34]:
print(current_directory)
print(kartaview_keys)

/workspace/workspace/ufo-prediction/image_data_NLD
                   lon        lat  age_right                     id
97  4583413   4.738666  52.947502     1953.0  v0.1-NLD.9.15_1-24152
315 19789747  5.570947  51.770518     1974.0   v0.1-NLD.8.47_1-6310
455 3874217   5.117115  52.237031     1972.0   v0.1-NLD.9.59_1-6586
411 22572960  5.554535  52.013209     1950.0  v0.1-NLD.11.28_1-9596
15  32521724  5.206793  52.362430     1982.0   v0.1-NLD.2.1_1-85419
...                ...        ...        ...                    ...
269 18053623  5.787166  51.682889     1935.0    v0.1-NLD.8.41_1-176
123 42905501  6.715116  52.105296     1980.0   v0.1-NLD.4.19_1-4362
127 25117464  6.904889  52.745651     1984.0   v0.1-NLD.1.6_1-23122
373 26904799  5.630706  53.043482     1989.0   v0.1-NLD.3.25_1-5671
262 37334101  5.668091  53.212675     1974.0   v0.1-NLD.3.18_1-9292

[100000 rows x 4 columns]


In [38]:
# Assuming current_directory and processed_df_path are set from the previous code chunk
print(f"Images will be saved in: {current_directory}")
print(f"Using kartaview_keys from: {processed_df_path}")

# Consistency check between image directory and kartaview_keys path
expected_csv_map = {
    '/workspace/workspace/ufo-prediction/image_data': '/workspace/workspace/ufo-prediction/demo/kartaview_key.csv',
    '/workspace/workspace/ufo-prediction/image_data_NLD': '/workspace/workspace/ufo-prediction/demo/kartaview_key_NLD.csv',
    '/workspace/workspace/ufo-prediction/image_data_FRA': '/workspace/workspace/ufo-prediction/demo/kartaview_key_FRA.csv',
    '/workspace/workspace/ufo-prediction/image_data_ESP': '/workspace/workspace/ufo-prediction/demo/kartaview_key_ESP.csv',
}

# Stop the code if using image_data directory
if current_directory == '/workspace/workspace/ufo-prediction/image_data':
    print("Download for the 'image_data' directory has already been completed. Stopping execution.")
    # Use `exit()` or `sys.exit()` depending on your environment
    exit()

if processed_df_path != expected_csv_map.get(current_directory):
    print("Inconsistency detected between the image directory and the kartaview_keys path. Please check.")
    exit()

image_count = {} 
print("Number of buildings remaining: ",len(kartaview_keys))

for index, row in kartaview_keys.iterrows():
    precision = 6  # Start with 6 decimal places
    success = False  # Flag to indicate if the request was successful

    while precision > 2 and not success:
        # Format lon and lat to the current precision
        lon = f"{row['lon']:.{precision}f}"
        lat = f"{row['lat']:.{precision}f}"

        # Construct the API URL
        url = "https://api.openstreetcam.org/2.0/photo/?lat={}&lng={}".format(lat, lon)

        # Send a GET request to the API
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()
            url_pattern = r'https://storage\d+\.openstreetcam\.org/files/photo/\d+/\d+/\d+/[^"]+\.jpg'
            urls = re.findall(url_pattern, json.dumps(data))

            filtered_urls = [
                url for url in urls
                if not any(x in url.rsplit('/', 2)[-2] for x in ["{{sizeprefix}}", "proc"]) and
                ("th" in url.rsplit('/', 2)[-2] and not "lth" in url.rsplit('/', 2)[-2])
            ]

            if filtered_urls:
                # Initialize or update the image count for the current ID
                building_id = row['id']  # Assuming 'id' column exists in your DataFrame
                if building_id not in image_count:
                    image_count[building_id] = 0

                for image_url in filtered_urls:
                    image_count[building_id] += 1  # Increment the image count for the building
                    subscript = image_count[building_id]  # Subscript for the file name
                    file_name = f"{row['age_right']}_{building_id}_{subscript}.jpg"
                    file_path = os.path.join(current_directory, file_name)

                    # Check if the file already exists
                    if os.path.exists(file_path):
                        print(f"File already exists: {file_path}. Skipping download.")
                    else:
                        image_response = requests.get(image_url)

                        if image_response.status_code == 200:
                            with open(file_path, 'wb') as f:
                                f.write(image_response.content)
                            print("Image downloaded successfully: {}".format(file_path))
                        else:
                            print("Failed to download the image.")
                success = True  # Mark success as True to exit the while loop
            else:
                print("No suitable images found for location: lon={}, lat={}".format(lon, lat))
                precision -= 1  # Reduce precision by one decimal place
        else:
            print("Failed to retrieve data from the API for location: lon={}, lat={}. Trying with reduced precision.")
            

    # After processing, remove the row from df_subset
    kartaview_keys = kartaview_keys.drop(index)

    # Save the updated DataFrame to a CSV file
    kartaview_keys.to_csv(processed_df_path, index=False)

    if not success:
        print("Unable to retrieve data from the API with sufficient precision for location: lon={}, lat={}".format(row['lon'], row['lat']))


Images will be saved in: /workspace/workspace/ufo-prediction/image_data_NLD
Using kartaview_keys from: /workspace/workspace/ufo-prediction/demo/kartaview_key_NLD.csv
Number of buildings remaining:  99951
No suitable images found for location: lon=5.939477, lat=52.210299
No suitable images found for location: lon=5.93948, lat=52.21030
No suitable images found for location: lon=5.9395, lat=52.2103
No suitable images found for location: lon=5.939, lat=52.210
Unable to retrieve data from the API with sufficient precision for location: lon=5.939476969762059, lat=52.21029913111886
No suitable images found for location: lon=5.742202, lat=53.159229
No suitable images found for location: lon=5.74220, lat=53.15923
No suitable images found for location: lon=5.7422, lat=53.1592
No suitable images found for location: lon=5.742, lat=53.159
Unable to retrieve data from the API with sufficient precision for location: lon=5.742202059394881, lat=53.15922871413438
No suitable images found for location: l

KeyboardInterrupt: 

In [14]:
# Iterate over each row in df_subset
for index, row in kartaview_keys.iterrows():
    # Format lon and lat to ensure they are strings with up to 8 decimal places
    lon = "{:.6f}".format(row['lon']).rstrip('0').rstrip('.')
    lat = "{:.6f}".format(row['lat']).rstrip('0').rstrip('.')

    # Check if lon and lat have less than 8 digits after the decimal
    if len(lon.split('.')[-1]) < 3 or len(lat.split('.')[-1]) < 3:
        print("* Location not specific enough")
        continue

    # Construct the API URL
    url = "https://api.openstreetcam.org/2.0/photo/?lat={}&lng={}".format(lat, lon)
    #url="http://api.openstreetcam.org/2.0/list/nearby-photos/?lat={}&lng={}".format(lat, lon)
    #print(url)

    # Send a GET request to the API
    response = requests.get(url)

    # Proceed if the request was successful
    if response.status_code == 200:
        data = response.json()

        # Define a regular expression pattern to match the URLs
        url_pattern = r'https://storage\d+\.openstreetcam\.org/files/photo/\d+/\d+/\d+/[^"]+\.jpg'

        # Find all matches of the pattern
        urls = re.findall(url_pattern, json.dumps(data))

        # Filter URLs based on the specified conditions
        filtered_urls = [url for url in urls if not any(x in url.rsplit('/', 2)[-2] for x in ["{{sizeprefix}}", "proc"]) and ("th" in url.rsplit('/', 2)[-2] and not "lth" in url.rsplit('/', 2)[-2])]

        # Download images from filtered URLs
        for image_url in filtered_urls:
            file_name = image_url.split('/')[-1]
            if not file_name.endswith('.jpg'):
                file_name += '.jpg'

            # Name the .jpg using age_right followed by the original file_name
            file_name = "{}_{}".format(row['age_right'], file_name)

            # Combine the directory with the filename
            file_path = os.path.join(current_directory, file_name)

            # Send a GET request to fetch the image
            image_response = requests.get(image_url)

            if image_response.status_code == 200:
                # Save the image to a file
                with open(file_path, 'wb') as f:
                    f.write(image_response.content)
                print("Image downloaded successfully: {}".format(file_path))
            else:
                print("Failed to download the image.")
        else:
            print("Failed to retrieve data from the API for location: lon={}, lat={}".format(lon, lat))
        # After processing, remove the row from df_subset
    kartaview_keys = kartaview_keys.drop(index)

    # Save the updated DataFrame to a CSV file
    kartaview_keys.to_csv(processed_df_path, index=False)


Failed to retrieve data from the API for location: lon=4.519975, lat=52.075455
Failed to retrieve data from the API for location: lon=5.970008, lat=50.885978
Failed to retrieve data from the API for location: lon=5.620564, lat=52.349998
Failed to retrieve data from the API for location: lon=4.540407, lat=52.060026
Failed to retrieve data from the API for location: lon=5.619979, lat=52.026556
Failed to retrieve data from the API for location: lon=4.700046, lat=52.295673
Failed to retrieve data from the API for location: lon=5.978112, lat=51.979975
Failed to retrieve data from the API for location: lon=5.07596, lat=52.070016
Failed to retrieve data from the API for location: lon=4.880037, lat=52.3599


KeyboardInterrupt: 

In [12]:

# Define the API endpoint with the specific latitude and longitude
# 13.38953209
base_url = "https://api.openstreetcam.org/2.0/photo/?lat={}&lng={}"


# Send a GET request to the API
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Debug: Print the response JSON to find the correct key for the image URL
    #print(response.json())

    data = response.json()

    # Define a regular expression pattern to match the URLs
    url_pattern = r'https://storage\d+\.openstreetcam\.org/files/photo/\d+/\d+/\d+/[^"]+\.jpg'

    # Find all matches of the pattern
    urls = re.findall(url_pattern, json.dumps(data))
    #print(urls)

    # Filter URLs that include "{{sizeprefix}}", "proc", or "th" before the last "/" sign
    filtered_urls = []
    for url in urls:
    # Split the URL by "/" to get all segments
        parts = url.split('/')
    # Check if the segment before the last one matches any of the specified patterns
        if len(parts) > 1:  # Ensure there are enough parts to inspect
            segment_to_check = parts[-2]  # Get the segment before the last one
        # Check if the segment does not contain any of the specified patterns
            if not any(pattern in segment_to_check for pattern in ["{{sizeprefix}}", "proc"]) and (segment_to_check.endswith("th") and not segment_to_check.endswith("lth")):
                filtered_urls.append(url)
    
    print(filtered_urls)
    
    if filtered_urls:  
        # Check if image_url is not None
        for image_url in filtered_urls:
            print(image_url)
            file_name = image_url.split('/')[-1]
            if not file_name.endswith('.jpg'):
                file_name += '.jpg'
        # Combine the directory with the filename
            file_path = os.path.join(current_directory, file_name)
        # Send a GET request to fetch the image
            image_response = requests.get(image_url)
    
            if image_response.status_code == 200:
            # Save the image to a file
                with open(file_path, 'wb') as f:
                    f.write(image_response.content)
                print("Image downloaded successfully.")
            else:
                print("Failed to download the image.")
    else:
        print("Image URL was not found in the API response.")
else:
    print("Failed to retrieve data from the API.")



['https://storage2.openstreetcam.org/files/photo/2016/8/25/th/14682_72e79_57bf17a3d0fc7.jpg', 'https://storage6.openstreetcam.org/files/photo/2017/11/10/th/966533_dfcfa_13.jpg', 'https://storage6.openstreetcam.org/files/photo/2017/11/10/th/966541_6cb2b_25.jpg', 'https://storage13.openstreetcam.org/files/photo/2020/7/26/th/2818418_83ecb_5f1d8e994f5a5.jpg', 'https://storage13.openstreetcam.org/files/photo/2020/7/28/th/2827634_9008c_5f20513eb1ff1.jpg', 'https://storage13.openstreetcam.org/files/photo/2020/8/5/th/2865646_84ea2_5f2a2bff88b76.jpg', 'https://storage13.openstreetcam.org/files/photo/2020/8/5/th/2865742_9bb8d_5f2a3576d6d95.jpg', 'https://storage13.openstreetcam.org/files/photo/2020/8/15/th/2917218_41295_5f37ec1e71d2e.jpg', 'https://storage13.openstreetcam.org/files/photo/2020/8/23/th/2943810_b2100_5f41d7658e0a9.jpg', 'https://storage13.openstreetcam.org/files/photo/2020/10/28/th/3196014_31e7b_5f998cea88f8f.jpg', 'https://storage13.openstreetcam.org/files/photo/2020/10/29/th/3198